# Common analyses that you can use to check for landscape transience

*This lesson made by Simon M Mudd and last updated 18/09/2023*

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

The assumption for these notebooks is that you are working in a notable container.

So these steps are turned into comments using a `#` symbol before each line.

If you want to work on google colab you will need to delete the `#` symbols

In [ ]:
#!pip install lsdviztools &> /dev/null

In [ ]:
#!pip install -q condacolab
#import condacolab
#condacolab.install()

In [ ]:
#!mamba install -y gdal proj lsdtopotools &> /dev/null

## Part 1: Get data

**First step, copy your Opentopography API key into a text file called "my_OT_api_key.txt" and make sure it is in the same file system as this notebook.**

If you don't know how to do that got to the `basic_topography` section and read lesson 1.

Make sure we are using the latest version of `lsdviztools`

In [ ]:
!pip install lsdviztools --upgrade

Double check that the version is correct. For this lesson we need version 0.4.7 or above.

In [ ]:
import lsdviztools
lsdviztools.__version__

We also need to import some stuff

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

**Next step: make sure the filename of your API key matches the one below.** Filenames are case sensitive. So if your file is "my_OT_API_key.txt" this will not work in the code block below (because api needs to be lower case).

**Then update the latitude and longitude of the lower left and upper right corner of your study area. You can get a latitude and longitude by right clicking on google maps.

In [ ]:
lower_left = [36.990554387425014, -2.318307057720176]
upper_right = [37.23367133834253, -1.8425313329873874]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "rio_aguas"
source_name = "COP30"


Aguas_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
Aguas_DEM.print_parameters()
Aguas_DEM.download_pythonic()

## Part 2: Get a hillshade

We are going to get a hillshade since this is used in all of the plotting routines in `lsdviztools`.

*You only need to execute this code, you don't need to change anything!*

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the image. It will also save a png file.

In [ ]:
DataDirectory = "./"
Dataset_prefix = "RC"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=600, save_fig = True)


print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))